# PeMS Data Analysis - I-680 Corridor

This notebook combines PeMS traffic data from 2019-2025 for I-680 (both Northbound and Southbound) and visualizes VMT trends.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

## Load and Combine Data Files

For each year and direction (680N and 680S), we load:
- Regular file: VMT and Delay (V_t=60 or 40 depending on year)
- _40 file: Delay 40 and Productivity Loss 40

Note: Some 680S files may be missing for certain years.

In [ ]:
# Define the years and directions we're loading
years = range(2019, 2026)
directions = ['680N', '680S']
base_path = Path('.')

all_data = []

for direction in directions:
    for year in years:
        # Load regular file (contains VMT)
        regular_file = base_path / f'pems_output{year}_{direction}.xlsx'
        file_40 = base_path / f'pems_output{year}_{direction}_40.xlsx'
        
        # Check if files exist
        if not regular_file.exists():
            print(f"Skipping {regular_file} (file not found)")
            continue
        
        df_regular = pd.read_excel(regular_file)
        
        # Rename columns for consistency
        df_regular = df_regular.rename(columns={
            'VMT (Veh-Miles)': 'VMT',
            'Delay (V_t=60) (Veh-Hours)': 'Delay_60',
            'Delay (V_t=40) (Veh-Hours)': 'Delay_60',  # Some years use 40 threshold in regular file
            '# Lane Points': 'Lane_Points',
            '% Observed': 'Pct_Observed'
        })
        
        # Start with regular file columns
        df_merged = df_regular[['Month', 'VMT', 'Delay_60', 'Lane_Points', 'Pct_Observed']].copy()
        
        # Initialize delay columns as None
        df_merged['Delay_40'] = None
        df_merged['Productivity_Loss_40'] = None
        
        # Merge with _40 file if it exists and has expected columns
        if file_40.exists():
            df_40 = pd.read_excel(file_40)
            
            # Check if this file has the expected delay columns
            has_delay_40 = 'Delay (V_t=40) (Veh-Hours)' in df_40.columns
            has_prod_loss = 'Lost Prod (40) (lane-mi-hrs)' in df_40.columns
            has_month = 'Month' in df_40.columns
            
            if has_month and (has_delay_40 or has_prod_loss):
                df_40 = df_40.rename(columns={
                    'Delay (V_t=40) (Veh-Hours)': 'Delay_40',
                    'Lost Prod (40) (lane-mi-hrs)': 'Productivity_Loss_40',
                })
                
                # Build list of columns to merge
                merge_cols = ['Month']
                if has_delay_40:
                    merge_cols.append('Delay_40')
                if has_prod_loss:
                    merge_cols.append('Productivity_Loss_40')
                
                # Drop the placeholder columns before merge
                df_merged = df_merged.drop(columns=[c for c in ['Delay_40', 'Productivity_Loss_40'] if c in merge_cols])
                
                df_merged = df_merged.merge(
                    df_40[merge_cols],
                    on='Month',
                    how='left'
                )
            else:
                print(f"Warning: {file_40.name} has unexpected columns, skipping delay data")
        
        df_merged['Year'] = year
        df_merged['Direction'] = direction
        all_data.append(df_merged)
        
# Combine all years and directions
df = pd.concat(all_data, ignore_index=True)

# Ensure Month is datetime
df['Month'] = pd.to_datetime(df['Month'])

# Sort by direction and date
df = df.sort_values(['Direction', 'Month']).reset_index(drop=True)

print(f"Combined dataset shape: {df.shape}")
print(f"\nRecords per direction:")
print(df.groupby('Direction').size())
df.head()

In [3]:
# View the complete dataset
df.info()
print("\n")
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Month                 84 non-null     datetime64[ns]
 1   VMT                   84 non-null     float64       
 2   Delay_60              84 non-null     float64       
 3   Lane_Points           84 non-null     int64         
 4   Pct_Observed          84 non-null     float64       
 5   Delay_40              84 non-null     float64       
 6   Productivity_Loss_40  84 non-null     float64       
 7   Year                  84 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 5.4 KB




,Month,VMT,Delay_60,Lane_Points,Pct_Observed,Delay_40,Productivity_Loss_40,Year
count,84,8.400000e+01,84.000000,8.400000e+01,84.000000,84.000000,84.000000,84.000000
mean,2022-06-16 14:17:08.571428608,1.255921e+08,174781.586905,4.884846e+06,58.913095,104047.035714,1831.805952,2022.000000
min,2019-01-01 00:00:00,6.776629e+07,13218.100000,4.418355e+06,32.000000,13218.100000,209.100000,2019.000000
25%,2020-09-23 12:00:00,1.189327e+08,112954.250000,4.750056e+06,55.650000,58982.525000,1123.575000,2020.000000
50%,2022-06-16 00:00:00,1.277917e+08,160807.100000,4.900638e+06,59.050000,105255.650000,1851.150000,2022.000000
75%,2024-03-08 18:00:00,1.379875e+08,237120.175000,5.061399e+06,61.925000,134111.675000,2406.600000,2024.000000
max,2025-12-01 00:00:00,1.482757e+08,372857.800000,5.171280e+06,75.900000,240433.700000,3678.400000,2025.000000
std,NaN,1.519397e+07,89505.214788,1.656416e+05,7.446771,53980.379952,841.344062,2.012012


## VMT Comparison: 680N vs 680S (2019-2025)

In [ ]:
# Create side-by-side bar charts for 680N and 680S
fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=False)

# Color palette by year
year_colors = plt.cm.viridis([i/6 for i in range(7)])

for idx, direction in enumerate(['680N', '680S']):
    ax = axes[idx]
    df_dir = df[df['Direction'] == direction].copy()
    
    if len(df_dir) == 0:
        ax.text(0.5, 0.5, f'No data available for {direction}', 
                ha='center', va='center', transform=ax.transAxes, fontsize=14)
        ax.set_title(f'I-{direction} Monthly VMT', fontsize=14, fontweight='bold')
        continue
    
    # Create month labels
    df_dir['Month_Label'] = df_dir['Month'].dt.strftime('%Y-%m')
    
    # Create bar chart
    bars = ax.bar(range(len(df_dir)), df_dir['VMT'] / 1e6, width=0.8, edgecolor='black', linewidth=0.5)
    
    # Color bars by year
    for i, bar in enumerate(bars):
        year_idx = df_dir.iloc[i]['Year'] - 2019
        bar.set_facecolor(year_colors[year_idx])
    
    # Set labels
    ax.set_ylabel('VMT (Millions of Vehicle-Miles)', fontsize=12)
    ax.set_title(f'I-{direction} Monthly Vehicle Miles Traveled (2019-2025)', fontsize=14, fontweight='bold')
    
    # Set x-axis ticks (show every 6 months)
    tick_positions = range(0, len(df_dir), 6)
    tick_labels = [df_dir.iloc[i]['Month_Label'] for i in tick_positions]
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels, rotation=45, ha='right')
    
    # Add gridlines
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    ax.set_axisbelow(True)

# Add shared legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=year_colors[i], label=str(2019+i)) for i in range(7)]
fig.legend(handles=legend_elements, title='Year', loc='upper right', bbox_to_anchor=(0.99, 0.99))

axes[-1].set_xlabel('Month', fontsize=12)

plt.tight_layout()
plt.savefig('vmt_680_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("Chart saved as 'vmt_680_comparison.png'")

In [ ]:
# Overlay line chart comparing 680N vs 680S VMT
fig, ax = plt.subplots(figsize=(16, 8))

colors = {'680N': '#1f77b4', '680S': '#ff7f0e'}
linestyles = {'680N': '-', '680S': '--'}

for direction in ['680N', '680S']:
    df_dir = df[df['Direction'] == direction].copy()
    if len(df_dir) == 0:
        continue
    ax.plot(df_dir['Month'], df_dir['VMT'] / 1e6, 
            label=f'I-{direction}', 
            color=colors[direction],
            linestyle=linestyles[direction],
            linewidth=2,
            marker='o',
            markersize=3)

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('VMT (Millions of Vehicle-Miles)', fontsize=12)
ax.set_title('I-680 VMT Comparison: Northbound vs Southbound (2019-2025)', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.yaxis.grid(True, linestyle='--', alpha=0.7)
ax.set_axisbelow(True)

# Rotate x-axis labels
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.savefig('vmt_680_overlay.png', dpi=150, bbox_inches='tight')
plt.show()

print("Chart saved as 'vmt_680_overlay.png'")

In [5]:
# Save the combined dataset
df.to_csv('pems_combined_data.csv', index=False)
print("Combined dataset saved as 'pems_combined_data.csv'")

Combined dataset saved as 'pems_combined_data.csv'
